In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import sqlite3
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up the driver
driver = webdriver.Chrome()

# Example URL - replace with the actual URL
url = "https://www.zillow.com/toronto-on/sold/"
driver.get(url)

input("Please solve the CAPTCHA manually and press Enter to continue...")

# Wait for the listings (article elements) to load
WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'article[data-test="property-card"]')))

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('zillow_data.db')
cursor = conn.cursor()

# Create a table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS listings (
        address TEXT,
        price TEXT,
        link TEXT,
        bedrooms TEXT,
        bathrooms TEXT,
        floor_size TEXT,
        property_type TEXT
    )
''')

# Find all the <article> elements containing property cards
listings = driver.find_elements(By.CSS_SELECTOR, 'article[data-test="property-card"]')

# Loop through each listing and extract details
for listing in listings:
    # Extract the address
    try:
        address = listing.find_element(By.CSS_SELECTOR, 'a[data-test="property-card-link"] address').text
    except:
        address = "Not available"
    
    # Extract the price
    try:
        price = listing.find_element(By.CSS_SELECTOR, 'span[data-test="property-card-price"]').text
    except:
        price = "Not available"
    
    # Extract the link to the property page
    try:
        link = listing.find_element(By.CSS_SELECTOR, 'a[data-test="property-card-link"]').get_attribute('href')
    except:
        link = "Not available"
    
    # Extract the number of bedrooms
    try:
        bedrooms = listing.find_element(By.XPATH, './/ul/li[1]/b').text
    except:
        bedrooms = "Not available"
    
    # Extract the number of bathrooms
    try:
        bathrooms = listing.find_element(By.XPATH, './/ul/li[2]/b').text
    except:
        bathrooms = "Not available"
    
    # Extract the floor size (if available)
    try:
        floor_size = listing.find_element(By.XPATH, './/div[@class="StyledPropertyCardHomeDetailsList-c11n-8-106-0__sc-1j0som5-0 eDOePJ"]/li[1]').text
    except:
        floor_size = "Not available"
    
    # Extract the property type (if available)
    try:
        property_type = listing.find_element(By.XPATH, './/div[contains(text(), "Type")]').text
    except:
        property_type = "Not available"
    
    # Insert the scraped data into the database
    cursor.execute('''
        INSERT INTO listings (address, price, link, bedrooms, bathrooms, floor_size, property_type)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (address, price, link, bedrooms, bathrooms, floor_size, property_type))

# Commit the transaction and close the connection
conn.commit()
conn.close()

driver.quit()

print("Data has been stored in the database.")

In [ ]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('zillow_data.db')

# Query the listings table
query = "SELECT * FROM listings"
df = pd.read_sql(query, conn)

# Display the data in a pandas DataFrame
df.head()

# Close the connection
conn.close()
